In [56]:
import tqdm
from loader_findata import finloader
import pandas as pd
import numpy as np
class windowed_learning_pipeline:
    def __init__(self, _pth:str, _train_size:int, _dropout_size:int, _win_size:int, _win_train_size:int):
        '''
        С помощью метода get_next() выдаёт следующее скользящее окно для обучения LSTM сети
        Аргументы:
        - pth: путь к 9 csv файлам с рыночными данными
        - test_percentage: процент от данных для тестовой выборки
        - dropout_percentage: процент данных, составляющих отступ между train и test
        - win_percentage: процент данных, которые составляют одно окно
        - win_train_percentage: процент данных для обучения, которые составляют одно окно
        - win_test_percentage: процент данных для тренировки, которые составляют одно окно
        '''

        self.train_size = _train_size
        self.data = finloader(_pth)
        self.length =len(self.data)
        self.dropout_size = _dropout_size
        self.win_size = _win_size
        self.win_train_size = _win_train_size
        self.getted_cnt = 0
        self.dropout_flag = 0
        self.columns = []

    def get_test(self):
        if not self.dropout_flag:
            return None
        else:
            df_test = pd.DataFrame()
            for i in range(self.getted_cnt, self.length): 
                df_test = pd.concat([df_test, self.data.step()], ignore_index=True)
            return df_test
    
    def gone_dropout(self):
        self.dropout_flag = 1
        for i in range(self.dropout_size):
            self.getted_cnt+=1
            self.data.step()
    
    def get_nxt(self):
        if self.getted_cnt + self.win_size >= self.train_size:
            if not self.dropout_flag:
                self.gone_dropout()
                return(None)
            else:
                return(None)
        
        cur_row = self.data.step()
        if self.columns == []:
            self.columns = cur_row.columns
        
        cur_row = cur_row.replace('', np.nan)
        cur_time = cur_row.index[0]
        np_row = cur_row.to_numpy(dtype=np.float32)

        train = np.zeros(dtype=np.float32, shape=(self.win_train_size, len(self.columns)))
        test = np.zeros(dtype=np.float32, shape=(self.win_size - self.win_train_size, len(self.columns)))

        train_time = ['' for i in range(self.win_train_size)]
        test_time = ['' for i in range(self.win_size - self.win_train_size)]

        print(np_row)
        for i in tqdm.tqdm(range(0, self.win_size)):
            if self.getted_cnt != 0:
                cur_row = self.data.step()
                cur_time = cur_row.index[0]
                np_row = cur_row.replace('', np.nan).index[0].to_numpy(dtype=np.float32)
            
            if i < self.win_train_size:
                train[i] = np_row
                train_time[i] = cur_time
            else:
                test[i - self.win_train_size] = np_row
                test_time[i - self.win_train_size] = cur_time

        
        self.getted_cnt += (self.win_size - self.win_train_size)

        print(len(train_time), train.shape)
        df_train = pd.DataFrame(index=train_time,
                                columns=self.columns,
                                data=train)
        
        df_test = pd.DataFrame(index=test_time,
                                columns=self.columns,
                                data=test)

        return (df_train, df_test)

In [59]:
lrn_pipe = windowed_learning_pipeline(
    _pth = "C:/Users/Student/UPS2025/UPS2025_FinMat/data/clear_data",
    _train_size = 300000,
    _dropout_size = 10000,
    _win_size = 10000,
    _win_train_size = 9000,
)

C:\Users\Student\AppData\Local\Temp\ipykernel_17712\2174390409.py:55: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  cur_row = cur_row.replace('', np.nan)


[[      nan       nan       nan ...       nan       nan 65295.164]]


100%|██████████| 10000/10000 [00:00<00:00, 195730.27it/s]

9000 (9000, 3681)
